In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import matplotlib.pyplot as plt
import tabulate as tb

In [6]:
TICKER = "AAPL"
START_DATE = "2000-01-01"
END_DATE = None
TIMEFRAME = "1d"    # 1d, 1h, 15m, 5m

In [7]:
df = yf.download(
    TICKER, 
    start=START_DATE,
    end=END_DATE,
    interval=TIMEFRAME,
    progress=False
)

if isinstance(df.columns, pd.MultiIndex):
    df.columns = [col[0] for col in df.columns]

df.dropna(inplace=True)
print(tb.tabulate(df.tail(), headers='keys', tablefmt='psql'))

C:\Users\BPX Michał Dziedziak\AppData\Local\Temp\ipykernel_20908\3251543518.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(


+---------------------+---------+--------+--------+--------+-------------+
| Date                |   Close |   High |    Low |   Open |      Volume |
|---------------------+---------+--------+--------+--------+-------------|
| 2025-10-15 00:00:00 |  249.34 | 251.82 | 247.47 | 249.49 | 3.38936e+07 |
| 2025-10-16 00:00:00 |  247.45 | 249.04 | 245.13 | 248.25 | 3.9777e+07  |
| 2025-10-17 00:00:00 |  252.29 | 253.38 | 247.27 | 248.02 | 4.9147e+07  |
| 2025-10-20 00:00:00 |  262.24 | 264.38 | 255.63 | 255.89 | 9.0483e+07  |
| 2025-10-21 00:00:00 |  262.77 | 265.29 | 261.83 | 261.88 | 4.6659e+07  |
+---------------------+---------+--------+--------+--------+-------------+


In [8]:
df["rsi"] = ta.rsi(df["Close"], length=14)
macd = ta.macd(df["Close"])
df = df.join(macd)

bbands = ta.bbands(df["Close"], length=20)
df = df.join(bbands)

df["ema_10"] = ta.ema(df["Close"], length=10)
df["ema_20"] = ta.ema(df["Close"], length=20)
df["ema_50"] = ta.ema(df["Close"], length=50)
df["atr"] = ta.atr(df["High"], df["Low"], df["Close"], length=14)
stoch = ta.stoch(df["High"], df["Low"], df["Close"])
df["stoch_k"] = stoch.iloc[:, 0]
df["stoch_d"] = stoch.iloc[:, 1]
df["returns"] = df["Close"].pct_change()
df['direction'] = np.where(df['returns'] > 0, 1, 0)

df.dropna(inplace=True)

print(tb.tabulate(df.head(), headers='keys', tablefmt='psql'))
print(tb.tabulate(df.tail(), headers='keys', tablefmt='psql'))

df.to_csv(f"./../data/{TICKER}.csv")

+---------------------+----------+----------+----------+----------+-------------+---------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+----------+----------+----------+-----------+-----------+-----------+------------+-------------+
| Date                |    Close |     High |      Low |     Open |      Volume |     rsi |   MACD_12_26_9 |   MACDh_12_26_9 |   MACDs_12_26_9 |   BBL_20_2.0_2.0 |   BBM_20_2.0_2.0 |   BBU_20_2.0_2.0 |   BBB_20_2.0_2.0 |   BBP_20_2.0_2.0 |   ema_10 |   ema_20 |   ema_50 |       atr |   stoch_k |   stoch_d |    returns |   direction |
|---------------------+----------+----------+----------+----------+-------------+---------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+----------+----------+----------+-----------+-----------+-----------+------------+-------